In [10]:
#import packages and data
import pandas as pd

df_tags = pd.read_csv('../data/raw/ml-25m/tags.csv')
df_gscores = pd.read_csv('../data/raw/ml-25m/genome-scores.csv')
df_gtags = pd.read_csv('../data/raw/ml-25m/genome-tags.csv')
df_links = pd.read_csv('../data/raw/ml-25m/links.csv')
df_movies = pd.read_csv('../data/raw/ml-25m/movies.csv')
df_ratings = pd.read_csv('../data/raw/ml-25m/ratings.csv')
df_ratings.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [13]:
##create large dataframe
#The same movieId refers to the same movie between `ratings.csv`, `tags.csv`, `movies.csv`, and `links.csv`
df = pd.merge(df_movies, df_links, how="inner", on=['movieId'])
display(df.describe())

df = pd.merge(df, df_tags, on=['movieId'], how='inner')

df = pd.merge(df, df_ratings, on=['movieId'], how='inner')
#display(df.describe())

,movieId,imdbId,tmdbId
count,62423.000000,6.242300e+04,62316.000000
mean,122220.387646,1.456706e+06,155186.689999
std,63264.744844,2.098007e+06,153362.694700
min,1.000000,1.000000e+00,2.000000
25%,82146.500000,8.168650e+04,36768.750000
50%,138022.000000,3.258050e+05,86750.500000
75%,173222.000000,2.063724e+06,255255.250000
max,209171.000000,1.117094e+07,646282.000000


MemoryError: Unable to allocate 81.1 GiB for an array with shape (10883253790,) and data type int64

MemoryError: Unable to allocate 81.1 GiB for an array with shape (10883253790,) and data type int64

##### Test Markdown
